In [ ]:
!pip install transformers datasets
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EvalPrediction
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from google.colab import drive
import torch

# Carregar os dados
drive.mount('/content/drive')
dataset = load_dataset('csv', data_files={
                                          'test': '/content/drive/MyDrive/Colab Notebooks/df_stj_teste.csv'
                                          })

# Tokenizador e modelo JurisBERT
tokenizer = AutoTokenizer.from_pretrained("alfaneo/jurisbert-base-portuguese-uncased")
model = AutoModelForSequenceClassification.from_pretrained("alfaneo/jurisbert-base-portuguese-uncased", num_labels=7)

Mounted at /content/drive


Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/254k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/482k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at alfaneo/jurisbert-base-portuguese-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
label_map_test = {'DIREITO ADMINISTRATIVO': 0,
                  'DIREITO CIVIL': 1,
                  'DIREITO DA CRIANÇA E DO ADOLESCENTE': 2,
                  'DIREITO DO CONSUMIDOR': 3,
                  'DIREITO EMPRESARIAL': 4,
                  'DIREITO PENAL': 5,
                  'DIREITO AMBIENTAL': 6
}

def preprocess_testing_data(examples):
    examples["label"] = [label_map_test[label] for label in examples["MATERIA"]]
    return tokenizer(examples["EMENTA"], padding="max_length", truncation=True, max_length=512)

encoded_dataset = {}

# Pré-processando o conjunto de teste
encoded_dataset["test"] = dataset["test"].map(preprocess_testing_data, batched=True)

# Função para calcular métricas
def compute_metrics(predictions, labels):
    preds = predictions.argmax(axis=-1)  # Classe predita
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    precision = precision_score(labels, preds, average="weighted", zero_division=0)
    recall = recall_score(labels, preds, average="weighted", zero_division=0)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

Map:   0%|          | 0/1111 [00:00<?, ? examples/s]

In [ ]:
# Avaliação no conjunto de teste
model.eval()
all_predictions = []
all_labels = []

with torch.no_grad():
    # Iterate over the items in encoded_dataset["test"]
    for example in encoded_dataset["test"]:
        # Prepara as entradas para o modelo
        inputs = {
            "input_ids": torch.tensor(example["input_ids"]).unsqueeze(0),
            "attention_mask": torch.tensor(example["attention_mask"]).unsqueeze(0)
        }
        # Gera logits
        outputs = model(**inputs)
        logits = outputs.logits
        all_predictions.append(logits.cpu().numpy())
        all_labels.append(example["label"])

# Converter para arrays e calcular métricas
all_predictions = np.vstack(all_predictions)
all_labels = np.array(all_labels)
metrics = compute_metrics(all_predictions, all_labels)

# Exibir resultados
print(f"Accuracy: {metrics['accuracy']}")
print(f"F1 Score: {metrics['f1']}")
print(f"Precision: {metrics['precision']}")
print(f"Recall: {metrics['recall']}")

Accuracy: 0.33123312331233123
F1 Score: 0.1894286773325567
Precision: 0.13270158385357564
Recall: 0.33123312331233123
